In [1]:
import math
import numpy as np
from source.msd_datasets import *
from source.utils import split_and_load
from transformers import BertTokenizer
from source.mb_masked_lm import MusicBertMaskedLM

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# REVIEWS DATASET
reviews_dataset = MSDDatasetReviewsMFCC(tokenizer = tokenizer)
train_dataloader, val_dataloader = split_and_load(reviews_dataset, workers=4, batch_size=8, split_size=0.02)

sound_features = 20 # MFCC
evals = 1

model = MusicBertMaskedLM(sound_features).cuda()
#model.load_pretrained()

In [2]:
model.pretrain_model(train_dataloader, None, epochs=6, eval_per_epoch=evals)

In [4]:
%matplotlib widget

evals = 1

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth

cls_loss = model.cls_loss_curve
sound_loss = [x/(2048) for x in model.sound_loss_curve]
token_loss = [x/16 for x in model.tokens_loss_curve]

cls_loss = smooth(model.cls_loss_curve, 200)[100:-100]
sound_loss = smooth(sound_loss, 500)[500:-500]
token_loss = smooth(token_loss, 10)[100:-100]

plot_curve(cls_loss, evals, color="red", metric_label="BCE")
plot_curve(token_loss, evals, color="blue", metric_label="NLL")
#plot_curve(sound_loss, evals, color="green", metric_label="MSE")


plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend(["Cls Loss", "Token Loss"]) #, "Sound Loss"

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Model BCE: 67.66%
Model NLL: 65.43%
Model MSE: 100.65%
